# 1. Basic Settings

In [1]:
import gensim
import os
import logging

# Log settings
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


c:\nlp\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
# Note that this imports ALL FILES in dir
class MySentences(object):

    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield gensim.utils.simple_preprocess(line)

# a memory - friendly iterator
sentences = MySentences('D:\\coca and GloWbE\\coca\\temp\\text2doc')



# 2. (skip) Preprocessing - Dividing text in file into Docs

Each file has hundreds of documents, with number tags leading the start of each article 
i.e) ##300000, ##300001, #300002 . . .

In [20]:
for i in range(1990, 2013): # reads each text at directory and prints a few lines
    file = open("D:\\COCA & GloWbE\\COCA\\texts\\text_academic_rpe\\w_acad_%d.txt" % i)
    newtext = file.read()
    print("File", str(i), ":", newtext[1:100], "\n")
    file.close()

File 1990 : ##4000161 Section : Life and Letters <p> I do n't remember hearing the phrase " white guilt " very  

File 1991 : ##4000171 Scientific fraud has become a very hot topic . Most scientists have traditionally believe 

File 1992 : ##4000003 Long-term data on community dynamics are a necessary part of research studies <p> Conserv 

File 1993 : ##4000011 Section : Features Animals and plants of the floodplain thrive , while river researchers  

File 1994 : ##4000031 Section : Features Grassroots efforts and the Constitution hold their own against religio 

File 1995 : ##4000208 A Scholar in Washington <p> I guess I was ready when the call came . Although hard at wor 

File 1996 : ##4000046 Section : Features A crane's-eye-view of Panama 's forest canopy yields biological surpri 

File 1997 : ##4000054 Section : Features Research sheds new light on the origin of humanity 's most intimate qu 

File 1998 : ##4000061 Section : Features Elegant experiments confirm long-held theory of

# Segement each yearly file to seperate documents

In [21]:
import re

doc_index = 1

for i in range(1990, 2013):
    file = open("D:\\COCA & GloWbE\\COCA\\texts\\text_academic_rpe\\w_acad_%d.txt" % i)
    newtext = file.read()
    
    # close file after reading
    file.close()
    
    text_ids = re.findall('\n##\d+', newtext)
    
    # Split file between " (line break) hash hash number+ "
    p = re.split('\n##\d+', newtext)
    p = p[1:] # due to list.split() the 1st item is blank
    p = list(zip(text_ids, p)) # zip text ids & text
    p = [''.join(item) for item in p ]
    
    print("File w_acad_%d" % i, "-", "Number of files : ", len(p))

    
    for f in p: # Writes out each element of list (=article) as a new file
        
        new_file = open(("D:\\COCA & GloWbe\\COCA\\Pre-processed text\\Doc2Vec Docs - Acad\\acad_%d.txt" % doc_index), 'w')
        new_file.write(f)

        doc_index += 1
        
print("Total number of files:", doc_index)

File w_acad_1990 - Number of files :  872
File w_acad_1991 - Number of files :  890
File w_acad_1992 - Number of files :  877
File w_acad_1993 - Number of files :  907
File w_acad_1994 - Number of files :  931
File w_acad_1995 - Number of files :  946
File w_acad_1996 - Number of files :  943
File w_acad_1997 - Number of files :  994
File w_acad_1998 - Number of files :  875
File w_acad_1999 - Number of files :  839
File w_acad_2000 - Number of files :  866
File w_acad_2001 - Number of files :  870
File w_acad_2002 - Number of files :  901
File w_acad_2003 - Number of files :  974
File w_acad_2004 - Number of files :  937
File w_acad_2005 - Number of files :  940
File w_acad_2006 - Number of files :  994
File w_acad_2007 - Number of files :  1157
File w_acad_2008 - Number of files :  912
File w_acad_2009 - Number of files :  1103
File w_acad_2010 - Number of files :  993
File w_acad_2011 - Number of files :  1054
File w_acad_2012 - Number of files :  451
Total number of files: 21227


# 3. Extracting Science-related article numbers

## 3.1 COCA excel sheet
- containins year, domain, date, author, etc. for each text  

In [22]:
import xlrd
workbook = xlrd.open_workbook('D:\COCA & GloWbE\COCA\coca_2015_12_23.xlsx')

In [24]:
# Print out sheet name
for i, sheet in enumerate(workbook.sheets()):
    print("Sheet",i+1 ,":" , sheet.name)

Sheet 1 : introduction
Sheet 2 : TOTALS
Sheet 3 : spoken
Sheet 4 : fiction
Sheet 5 : magazine
Sheet 6 : newspaper
Sheet 7 : academic
Sheet 8 : Sheet1


### Academic excel sheet 

In [33]:
acad_sheet = workbook.sheets()[6]

# Headers - column "domain" is at 4th col 
print(acad_sheet.row_values(0))

['textID', '# words', 'Year', 'Domain', 'Journal', 'Title', 'Author', 'PubInfo', 'Subject']


Our column of interest 'domain' is the 4th column. <br>
We iterate through each row and check the 4th column for "Sci/Tech", which has 4356 articles


In [29]:
sci_text_rows = []

for i in range(acad_sheet.nrows):
    
    row = acad_sheet.row_values(i)
    
    if("Sci/Tech" in row[3]): # row[3] == 'Domain'
        sci_text_rows.append(i)
#    print(mag_sheet.col_values(i)) # mag_sheet.col_values()

Row numbers of entries with science related topics

In [30]:
sci_text_rows

[770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 778,
 779,
 780,
 781,
 782,
 783,
 784,
 785,
 786,
 787,
 788,
 789,
 790,
 791,
 792,
 793,
 794,
 795,
 796,
 797,
 798,
 799,
 800,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 815,
 816,
 817,
 818,
 819,
 820,
 821,
 822,
 823,
 824,
 825,
 826,
 827,
 828,
 829,
 830,
 831,
 832,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 843,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 854,
 855,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 871,
 872,
 1663,
 1664,
 1665,
 1666,
 1667,
 1668,
 1669,
 1670,
 1671,
 1672,
 1673,
 1674,
 1675,
 1676,
 1677,
 1678,
 1679,
 1680,
 1681,
 1682,
 1683,
 1684,
 1685,
 1686,
 1687,
 1688,
 1689,
 1690,
 1691,
 1692,
 1693,
 1694,
 1695,
 1696,
 1697,
 1698,
 1699,
 1700,
 1701,
 1702,
 1703,
 1704,
 1705,
 1706,
 1707,
 1708,
 1709,
 1710,
 1711,
 1712,
 1713,
 1714,
 1715,
 1716,
 171

In [31]:
len(sci_text_rows)

4356

Put the relevant rows in a list

In [34]:
scienceAcadTexts_list = [acad_sheet.row_values(i) for i in sci_text_rows]

In [35]:
scienceAcadTexts_list

[[4018430.0,
  9566.0,
  1990.0,
  'Sci/Tech',
  'Armed Forces & Society (Transaction Publishers)',
  'Congressional Oversight of the Department of Defense: Reconsidering the Conventional Wisdom.',
  'Lindsay, James M.<SUP>1</SUP>',
  'Fall90, Vol. 17 Issue 1, p7-33, 27p',
  'BUREAUCRACY;LEGISLATIVE oversight;LEGISLATORS;UNITED States. Congress;UNITED States. Dept. of Defense'],
 [4018431.0,
  5616.0,
  1990.0,
  'Sci/Tech',
  'Armed Forces & Society (Transaction Publishers)',
  'B. H. Liddell Hart and the Creation of a Theory of War, 1919-1933.',
  'Swain, Richard M.<SUP>1</SUP>',
  'Fall90, Vol. 17 Issue 1, p35-51, 17p',
  'ARMED Forces;INDUSTRIAL efficiency;TACTICS;WAR'],
 [4018432.0,
  9492.0,
  1990.0,
  'Sci/Tech',
  'Armed Forces & Society (Transaction Publishers)',
  'The State, Capitalism, and World War II: The U.S. Case.',
  'Feagin, Joe R.<SUP>1</SUP><BR>Riddell, Kelly<SUP>2</SUP>',
  'Fall90, Vol. 17 Issue 1, p53-79, 27p',
  'ADMINISTRATIVE agencies;AUTONOMY (Political scie

Check if any rows have different lengths with each other - that is, if they have differing number of elements

In [47]:
temp = [str(i) for i in scienceAcadTexts_list[0]]
temp_string = '\n'.join(temp)
print("A normal row would have %d elements, as: \n\n" % len(temp), temp_string)

A normal row would have 9 elements, as: 

 4018430.0
9566.0
1990.0
Sci/Tech
Armed Forces & Society (Transaction Publishers)
Congressional Oversight of the Department of Defense: Reconsidering the Conventional Wisdom.
Lindsay, James M.<SUP>1</SUP>
Fall90, Vol. 17 Issue 1, p7-33, 27p
BUREAUCRACY;LEGISLATIVE oversight;LEGISLATORS;UNITED States. Congress;UNITED States. Dept. of Defense


In [55]:
for i in range(len(scienceAcadTexts_list)):
    
    if i == 0: 
        temp = len(scienceAcadTexts_list[i])
        pass
    
    print(temp == len(scienceAcadTexts_list[i]))
    
    temp = len(scienceAcadTexts_list[i])
    

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Now to write out a new excel file with only the relevant entries

In [56]:
import xlsxwriter
processed_workbook = xlsxwriter.Workbook(r'D:\COCA & GloWbE\COCA\COCA Science texts Acad')
worksheet = processed_workbook.add_worksheet()

In [57]:
original_sheet_COLS = len(acad_sheet.row_values(0))
target_sheet_ROWS = len(scienceAcadTexts_list)
target_sheet_COLS = len(scienceAcadTexts_list[0])

# Write header
for col in range(original_sheet_COLS):
    worksheet.write(0, col, acad_sheet.row_values(0)[col])
    
# Write each row by iterating column-wise

for row in range(target_sheet_ROWS):
    for col in range(target_sheet_COLS):
        worksheet.write(row+1, col, scienceAcadTexts_list[row][col]) # row+1 because of the header
    
    

In [58]:
processed_workbook.close()